# Examining the Effects of DP on Exponential Smoothing Forecast Accuracy

***

In [1]:
# general modules
import pandas as pd
import numpy as np
import sktime

# import exponential smoothing forecasting model
from sktime.forecasting.exp_smoothing import ExponentialSmoothing

# functions for transformation+forecasting pipeline
from sktime.forecasting.compose import TransformedTargetForecaster

# time series transformations
from sktime.transformations.series.detrend import ConditionalDeseasonalizer

##### the `helper_functions.py` file contains many custom functions we wrote to aid in our analysis
##### `full_coding_analysis` combines all of the following - train-test split data,
##### data protection, train models, compare accuracies, return accuracy results
from helper_functions import *

# suppress warnings from exponential smoothing model not converging
import warnings
warnings.filterwarnings('ignore')


# nice time series plots
from sktime.utils.plotting import plot_series

In [2]:
# import weekly finance time series
Y = np.genfromtxt("../../Data/Train/Clean/weekly_finance_clean.csv", delimiter = ',', skip_header = 1)
Y = pd.DataFrame(Y)

In [3]:
# detrender = Detrender()
# detrended_series = [detrender.fit_transform(series) for _ , series in Y.iterrows()]
# detrended_series = [i+np.abs(np.min(i))+1.0 for i in detrended_series]
# Y = pd.concat(detrended_series, axis=1).T

***

## SES

In [4]:
# define forecasting model
# perform additive deseasonalization conditional on autocorrelation test for seasonality

forecaster = TransformedTargetForecaster(
    [
        ("forecast", ExponentialSmoothing(use_boxcox=False)),
    ]
)

In [5]:
results_dict_ses = {}
fcasts_ses = {}
fcasts_protected_ses = {}
tests = {}
epsilons = [1, 10, 20]
horizons = [1, 20]

In [6]:
for e in epsilons:
    for h in horizons:
        idx = "h="+str(h)+", epsilon = "+str(e)
        results_dict_ses[idx], tests[idx], fcasts_ses[idx], fcasts_protected_ses[idx] = full_coding_analysis(time_series_data=Y, 
                                                                                                             forecasting_model=forecaster, 
                                                                                                             forecast_horizon=h,
                                                                                                             epsilon=e)

In [7]:
results_dict_ses

{'h=1, epsilon = 1': {'Mean Accuracies': array([55.3207, 55.3207]),
  'Protected Mean Accuracies:': array([327.2314, 327.2314]),
  '% Change Mean accuracy:': array([-4.9152, -4.9152]),
  '% Change Median accuracy:': array([-5.7566, -5.7566]),
  '% Forecasted Points adjusted downward:': 0.5671,
  '% Forecasted Points adjusted upward:': 0.4329,
  '% Series with improved accuracy:': array([0.1341, 0.1341]),
  '% Series with reduced accuracy:': array([0.8659, 0.8659]),
  'Original Mean Absolute Error Upward Adjusted:': 67.2469,
  'Original Mean Absolute Error Downward Adjusted:': 46.2157,
  'Protected Mean Absolute Error Upward Adjusted:': 413.0976,
  'Protected Mean Absolute Error Downward Adjusted:': 261.6777},
 'h=20, epsilon = 1': {'Mean Accuracies': array([112.5495, 135.001 ]),
  'Protected Mean Accuracies:': array([329.759 , 347.0677]),
  '% Change Mean accuracy:': array([-1.9299, -1.5709]),
  '% Change Median accuracy:': array([-1.7699, -1.4732]),
  '% Forecasted Points adjusted dow

***
***

In [8]:
original_forecasts = fcasts_ses['h=20, epsilon = 20']
protected_forecasts = fcasts_protected_ses['h=20, epsilon = 20']
test = tests['h=20, epsilon = 20']

In [9]:
adjusted_up = original_forecasts < protected_forecasts
adjusted_up = pd.concat([row for i, row in adjusted_up.iterrows()])
adjusted_down = original_forecasts > protected_forecasts
adjusted_down = pd.concat([row for i, row in adjusted_down.iterrows()])

In [10]:
absolute_error_original = np.absolute(test - original_forecasts)
absolute_error_protected = np.absolute(test - protected_forecasts)

In [11]:
improved = absolute_error_original > absolute_error_protected
improved = pd.concat([row for i, row in improved.iterrows()])
worsened = absolute_error_original < absolute_error_protected
worsened = pd.concat([row for i, row in worsened.iterrows()])

In [12]:
np.mean(adjusted_down[improved])

0.42681992337164754

In [13]:
np.mean(adjusted_up[improved])

0.5731800766283525

***
***

## DES

In [14]:
# define forecasting model
# perform additive deseasonalization conditional on autocorrelation test for seasonality

forecaster = TransformedTargetForecaster(
    [
        ("forecast", ExponentialSmoothing(trend="additive", use_boxcox=False)),
    ]
)

In [15]:
results_dict_des = {}
fcasts_des = {}
fcasts_protected_des = {}
tests = {}
epsilons = [1, 10, 20]
horizons = [1, 20]

In [16]:
for e in epsilons:
    for h in horizons:
        idx = "h="+str(h)+", epsilon = "+str(e)
        results_dict_des[idx], tests[idx], fcasts_des[idx], fcasts_protected_des[idx] = full_coding_analysis(time_series_data=Y, 
                                                                                                             forecasting_model=forecaster, 
                                                                                                             forecast_horizon=h,
                                                                                                             epsilon=e)

In [17]:
results_dict_des

{'h=1, epsilon = 1': {'Mean Accuracies': array([54.4492, 54.4492]),
  'Protected Mean Accuracies:': array([479.9404, 479.9404]),
  '% Change Mean accuracy:': array([-7.8145, -7.8145]),
  '% Change Median accuracy:': array([-12.9318, -12.9318]),
  '% Forecasted Points adjusted downward:': 0.561,
  '% Forecasted Points adjusted upward:': 0.439,
  '% Series with improved accuracy:': array([0.0732, 0.0732]),
  '% Series with reduced accuracy:': array([0.9268, 0.9268]),
  'Original Mean Absolute Error Upward Adjusted:': 80.4633,
  'Original Mean Absolute Error Downward Adjusted:': 34.0903,
  'Protected Mean Absolute Error Upward Adjusted:': 652.2773,
  'Protected Mean Absolute Error Downward Adjusted:': 345.068},
 'h=20, epsilon = 1': {'Mean Accuracies': array([109.1455, 132.0211]),
  'Protected Mean Accuracies:': array([737.9296, 766.1162]),
  '% Change Mean accuracy:': array([-5.761, -4.803]),
  '% Change Median accuracy:': array([-5.0304, -4.1763]),
  '% Forecasted Points adjusted downwa

***
***

## TES

In [18]:
# define forecasting model
# perform additive deseasonalization conditional on autocorrelation test for seasonality

forecaster = TransformedTargetForecaster(
    [
        ("forecast", ExponentialSmoothing(trend="additive",
                                          seasonal="additive",
                                          sp=52,
                                          damped_trend=False, 
                                          use_boxcox=False)),
    ]
)

In [19]:
results_dict_tes = {}
fcasts_tes = {}
fcasts_protected_tes = {}
tests = {}
epsilons = [1, 10, 20]
horizons = [1, 20]

In [20]:
for e in epsilons:
    for h in horizons:
        idx = "h="+str(h)+", epsilon = "+str(e)
        results_dict_tes[idx], tests[idx], fcasts_tes[idx], fcasts_protected_tes[idx] = full_coding_analysis(time_series_data=Y, 
                                                                                                             forecasting_model=forecaster, 
                                                                                                             forecast_horizon=h,
                                                                                                             epsilon=e)

In [21]:
results_dict_tes

{'h=1, epsilon = 1': {'Mean Accuracies': array([66.4076, 66.4076]),
  'Protected Mean Accuracies:': array([951.7207, 951.7207]),
  '% Change Mean accuracy:': array([-13.3315, -13.3315]),
  '% Change Median accuracy:': array([-14.3371, -14.3371]),
  '% Forecasted Points adjusted downward:': 0.5793,
  '% Forecasted Points adjusted upward:': 0.4207,
  '% Series with improved accuracy:': array([0.061, 0.061]),
  '% Series with reduced accuracy:': array([0.939, 0.939]),
  'Original Mean Absolute Error Upward Adjusted:': 67.0527,
  'Original Mean Absolute Error Downward Adjusted:': 65.9391,
  'Protected Mean Absolute Error Upward Adjusted:': 1083.6856,
  'Protected Mean Absolute Error Downward Adjusted:': 855.8725},
 'h=20, epsilon = 1': {'Mean Accuracies': array([116.7834, 138.5197]),
  'Protected Mean Accuracies:': array([ 969.2088, 1216.407 ]),
  '% Change Mean accuracy:': array([-7.2992, -7.7815]),
  '% Change Median accuracy:': array([ -9.0962, -10.7079]),
  '% Forecasted Points adjuste